# Prediction: 
This project is divided in two. The first part is about completion (prediction) of words via some context or none. And the second part is dedicated to categorization of sentences for sentiment analysis.

It involves the use of:

- Maximum likelihood estimation (MLE)
- MLE with laplace smoothing
- Test set perplexity
* Bag of words
* Binary Naive Bayes

In [20]:
from nltk.corpus.reader import BracketParseCorpusReader
from nltk.lm import Vocabulary
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import bigrams
import numpy as np

corpus = BracketParseCorpusReader(root="corpora", fileids=["p1_train.txt"])
words = corpus.words()
sentences = corpus.sents()
parsed_texts = corpus.parsed_sents()
test = BracketParseCorpusReader(root="corpora", fileids=["p1_test.txt"])
test_set = test.sents()
vocab = Vocabulary(words, unk_cutoff=3)

### Little checks over the corpus

In our vocabulary, we have put all the words of the corpus that appear at least 3 times. Other present words are replace by the token \<UNK> as of every other words not present in the corpus.

In [21]:
word_num = []
for word in sorted(vocab)[-10:]:
    word_num.append((word, vocab[word]))
print(f"The 10 last words of the vocabulary with their number of appearence in the corpus:\n  {word_num}")

The 10 last words of the vocabulary with their number of appearence in the corpus:
  [('wry', 6), ('year', 32), ('years', 8), ('yes', 3), ('yet', 11), ('you', 166), ('young', 12), ('your', 31), ('yourself', 5), ('zippy', 3)]


In [22]:
count_unk = 0
for word in words:
    if word not in vocab:
        count_unk += 1
oov = (count_unk/len(words))*100

print(f"The number of unkown words: {count_unk}")
print(f"Out of vocabulary (OOV) rate: {round(oov, 3)}%".format())

The number of unkown words: 7352
Out of vocabulary (OOV) rate: 19.295%


In [23]:
sentence_10_bi = list(bigrams(list(pad_both_ends(sentences[9], n=2))))
print(f"All the bigrams of the 10th sentence:\n  {sentence_10_bi}")

All the bigrams of the 10th sentence:
  [('<s>', 'It'), ('It', "'s"), ("'s", 'a'), ('a', 'coming-of-age'), ('coming-of-age', 'story'), ('story', 'we'), ('we', "'ve"), ("'ve", 'all'), ('all', 'seen'), ('seen', 'bits'), ('bits', 'of'), ('of', 'in'), ('in', 'other'), ('other', 'films'), ('films', '--'), ('--', 'but'), ('but', 'it'), ('it', "'s"), ("'s", 'rarely'), ('rarely', 'been'), ('been', 'told'), ('told', 'with'), ('with', 'such'), ('such', 'affecting'), ('affecting', 'grace'), ('grace', 'and'), ('and', 'cultural'), ('cultural', 'specificity'), ('specificity', '.'), ('.', '</s>')]


### Text completion part

#### Maximum-likelihood estimation (MLE)

The probabilty of having a word (w) if we know the history (h) (context).

Since we don't know the probability, we estimate it by counting over the corpus.

The counting function is noted $C(*)$

$\^P(w|h) = \frac{C(h,w)}{C(h)}$

(if the $C(h) <= 0$ then $\^P(w|h) = 0$)

In [24]:
# This cell is needed to compute both mle and mle_laplace

# bi_count == C(h,w)
bi_count = {}
# mle will be == P(w|h) after computation, same for mle_laplace with the smoothing
# Not computed here, only initialised
mle = {}
mle_laplace = {}
for sentence in range(len(sentences)):
    for bi in list(bigrams(list(pad_both_ends(sentences[sentence], n=2)))):
        if bi[0] == '<s>':
            bi = (bi[0], vocab.lookup(bi[1]))
        elif bi[1] == '</s>':
            bi = (vocab.lookup(bi[0]), bi[1])
        else:
            bi = (vocab.lookup(bi[0]), vocab.lookup(bi[1]))
        if bi_count.get(bi[0], -1) == -1:
            bi_count[bi[0]] = {}
            mle[bi[0]] = {}
            mle_laplace[bi[0]] = {}
        count_bi = bi_count[bi[0]].get(bi[1], 0)
        bi_count[bi[0]][bi[1]] = count_bi+1

# h_count == C(h)
h_count = {}
for first_word in bi_count.keys():    
    count_next = 0
    for i in bi_count[first_word]:
        count_next += bi_count[first_word][i]
    h_count[first_word] = count_next

In [25]:
# Compute mle

best_nexts_key_mle = {}
for first_word in bi_count.keys():    
    for second_word in bi_count[first_word].keys():
        previous = bi_count[first_word][second_word]
        mle[first_word][second_word] = previous/h_count[first_word]
        
    key_next = list(mle[first_word].keys())
    key_next = sorted(key_next, key= mle[first_word].__getitem__)
    best_nexts_key_mle[first_word] = key_next   

def complete_next_word_mle(target):
    ret = {}
    nexts = best_nexts_key_mle[target]
    for i in range(5):
        ret[nexts[-i-1]] = round(mle[target][nexts[-i-1]], 4)
    return ret

In [26]:
# Run example

print("The 5 most probable completion after a target word (<s> is the start of a sentence):\n")
for target in ["<s>", "love"]:
    print(f"After the target {target}:")
    for word, proba in complete_next_word_mle(target).items():
        print(f"- {word}: {proba}")

The 5 most probable completion after a target word (<s> is the start of a sentence):

After the target <s>:
- <UNK>: 0.268
- The: 0.1105
- A: 0.0915
- It: 0.075
- This: 0.0345
After the target love:
- story: 0.1923
- <UNK>: 0.1538
- ,: 0.1154
- and: 0.0769
- with: 0.0769


#### MLE with Laplace smoothing

The approximation is smoothed via the following formula:

$\^P(w|h) = \frac{C(h,w) + 1}{C(h) + |V|}$

$|V|$ being the size of the vocabulary. More precisely in the code, it's the size of the vocabulary plus 3 tokens: 
- \<s> is the start of sentence token
- \</s> is the end of sentence token
- \<UNK> is the token for every word outside of the vocabulary (less than 3 occurence in the corpus)

In [27]:
# Compute mle_laplace (smoothed)

best_nexts_key_mle_laplace = {}
V = len(vocab) + 3
for first_word in bi_count.keys():
    for second_word in bi_count[first_word].keys():
        previous = bi_count[first_word][second_word]
        mle_laplace[first_word][second_word] = (previous + 1) / (h_count[first_word] + V)

    key_next = list(mle_laplace[first_word].keys())
    key_next = sorted(key_next, key= mle_laplace[first_word].__getitem__)
    best_nexts_key_mle_laplace[first_word] = key_next

def complete_next_word_mle_laplace(target):
    ret = {}
    nexts = best_nexts_key_mle_laplace[target]
    for i in range(5):
        ret[nexts[-i-1]] = round(mle_laplace[target][nexts[-i-1]], 4)
    return ret

# {'<UNK>': 0.14486107364445644, 'The': 0.05988670083625573, 'A': 0.04963582411653628}

In [28]:
# Run example

print("The 5 most probable completion after a target word (<s> is the start of a sentence) via the laplace smooth:\n")
for target in ["<s>", "love"]:
    print(f"After the target {target}:")
    for word, proba in complete_next_word_mle_laplace(target).items():
        print(f"- {word}: {proba}")

The 5 most probable completion after a target word (<s> is the start of a sentence) via the laplace smooth:

After the target <s>:
- <UNK>: 0.1449
- The: 0.0599
- A: 0.0496
- It: 0.0407
- This: 0.0189
After the target love:
- story: 0.0035
- <UNK>: 0.0029
- ,: 0.0023
- and: 0.0017
- with: 0.0017


#### Test set perplexity

The lower it is, the better. Because better model are more predictive and less uniformly random.

It can be interpreted as the number of words among which one predicts, if one would predict uniformly at random.

$|V| = 10000$ and $PP = 100$ means that the uncertainty is the same as if one would predict only among 100 possibilities with an equal probability of $\frac{1}{100}$ for each word.

The model actually predicts among 10'000 possibilities but with unequal probabilities.

In [29]:
# Care we use the test_set corpus here
# So some words may not be present in the training but do here
# Compute using 1/(h_count[bi[0]] + V)))

mult_score = 0
M = 0
for sentence in range(len(test_set)):
    for bi in list(bigrams(list(pad_both_ends(test_set[sentence], n=2)))):
        if bi[0] == '<s>':
            bi = (bi[0], vocab.lookup(bi[1]))
        elif bi[1] == '</s>':
            bi = (vocab.lookup(bi[0]), bi[1])
        else:
            bi = (vocab.lookup(bi[0]), vocab.lookup(bi[1]))
        mult_score += np.log2(mle_laplace[bi[0]].get(bi[1], 
                                                     1/(h_count[bi[0]] + V)))
        M += 1
        
LL = mult_score/M
PP = 2**(-LL)
print("Test set perplexity PP: {:.3f}".format(PP))

Test set perplexity PP: 167.333


### Text Categorization

The task of assigning a label or a category to an entire text or document. Here it is aim at sentiment analysis.

#### Bag of Words

Representation of a text document in an unordered set of words with their frequency in the document.

There are used for counting efficiently their appearence in the corpus.

#### Binary Naive Bayes

The formula for the Naive Bayes decision rule:

$C_{NB} = argmax_{c \in C} [log P(c) + \sum_i log P(w_i | c)]$

With

$\^P(c) = \frac{N_c}{N_{doc}}$

$\^P(w_i | c) = \frac{n_{w_i, c} + 1}{\sum_{w \in V} n_{w, c} + |V|}$

With:
- $N_c$ the number of doc labeled of class $c$ in training set
- $N_{doc}$ the number of doc in total in training set
- $n_{w_i, c}$ the number of occurences of word $w_i$ in all documents labeled $c$
- V the vocabulary size (no \<UNK> here because they are ignored from the bag of words)

In [30]:
# Preparation of the variable for the computation on the train corpus
N_neg = 0 # Nc1
N_pos = 0 # Nc2
N_doc = len(parsed_texts) #Ndoc
bag_neg = {} # nwi,c1
bag_pos = {} # nwi,c2
count_neg = 0 # Sum (nw,c1)
count_pos = 0 # Sum (nw,c2)
for sentence in range(N_doc):
    sequence = parsed_texts[sentence]
    if int(sequence.label()) < 2:
        N_neg += 1
        for word in sequence.leaves():
            count_neg += 1
            bag_neg[word] = bag_neg.get(word, 0) + 1
    else:
        N_pos += 1
        for word in sequence.leaves():
            count_pos += 1
            bag_pos[word] = bag_pos.get(word, 0) + 1

In [31]:
# Verification / testing on the test corpus
test_pred = [0]*len(test_set)
vocab_len = len(vocab) + 2
for sentence in range(len(test_set)):
    neg_score = 0
    pos_score = 0
    for word in test_set[sentence]:
        if word in vocab:
            # Sum log P(wi|c1)
            neg_score += np.log( (bag_neg.get(word, 0) + 1) / (count_neg + vocab_len) )
            pos_score += np.log( (bag_pos.get(word, 0) + 1) / (count_pos + vocab_len) )
    
    neg_score += np.log((N_neg/N_doc))    # Adding the prior log(P(c1))
    pos_score += np.log((N_pos/N_doc))
    # Pick the category that has the highest value
    if neg_score > pos_score:
        test_pred[sentence] = 0
    else:
        test_pred[sentence] = 1

test_verif = test.parsed_sents()
ratio = 0
for sentence in range(len(test_set)):
    y = int(test_verif[sentence].label())
    if (test_pred[sentence] == (y>=2)):
        ratio += 1
print((ratio/len(test_set))*100)

64.0


#### Binary Naive Bayes with small modification

Next we will introduces a list (words_seen) to keep track of words already counted to avoid double counting within the same sentence. This modification is as an attempt to capture the "bag-of-words" characteristics more accurately, especially if the same word occurs multiple times within the same sentence. Trying to be less biased by the corpus where some words may be overly used in the same sentence at some point.

All the comments from the code above is applicable below since it is mostly the same code.

In [32]:
N_neg = 0
N_pos = 0
N_doc = len(parsed_texts)
bag_neg = {}
bag_pos = {}
count_neg = 0
count_pos = 0
for sentence in range(N_doc):
    sequence = parsed_texts[sentence]
    words_seen = []
    if int(sequence.label()) < 2:
        N_neg += 1
        for word in sequence.leaves():
            if word not in words_seen:
                count_neg += 1
                bag_neg[word] = bag_neg.get(word, 0) + 1
                words_seen.append(word)
    else:
        N_pos += 1
        for word in sequence.leaves():
            if word not in words_seen:
                count_pos += 1
                bag_pos[word] = bag_pos.get(word, 0) + 1
                words_seen.append(word)

In [33]:
test_pred = [0]*len(test_set)
vocab_len = len(vocab) + 2
for sentence in range(len(test_set)):
    neg_score = 0
    pos_score = 0
    words_seen = []
    for word in test_set[sentence]:
        if word in vocab and word not in words_seen:
            neg_score += np.log( (bag_neg.get(word, 0) + 1) / (count_neg + vocab_len) )
            pos_score += np.log( (bag_pos.get(word, 0) + 1) / (count_pos + vocab_len) )
            words_seen.append(word)
    
    neg_score += np.log((N_neg/N_doc))
    pos_score += np.log((N_pos/N_doc))
    if neg_score > pos_score:
        test_pred[sentence] = 0
    else:
        test_pred[sentence] = 1

test_verif = test.parsed_sents()
ratio = 0
for sentence in range(len(test_set)):
    y = int(test_verif[sentence].label())
    if (test_pred[sentence] == (y>=2)):
        ratio += 1
print((ratio/len(test_set))*100)

66.5


#### Binary Naive Bayes with negation modification

Next we do an additionnal modification of the code to even more categorize the words within a negative sentence from a positive one. Introducing the modification of words using the following rule:

If we encounter a negative word (i.e "n't", "not", "no", "never"), then the next words till a punctuation are transformed: "word" --> "word_NOT"

The goal here is to distinguish the use a word that could be negate. For example, take the following sentences: "You're beautiful" and "You're not beautiful", as we can see beautifull (a positive words) can be present in negative sentence as well as positive ones, the meaning is inversed because of the not.

These transformation are then used in the training with the bags and the count.

In [34]:
negate = ["n't", "not", "no", "never"]
punct = ['.', ',', ':', '?', '!']

N_neg = 0
N_pos = 0
N_doc = len(parsed_texts)
bag_neg = {}
bag_pos = {}
count_neg = 0
count_pos = 0
neg_words_seen = set()
for sentence in range(N_doc):
    sequence = parsed_texts[sentence]
    in_neg = False
    if int(sequence.label()) < 2:
        N_neg += 1
        for word in sequence.leaves():
            if word in punct: in_neg = False
            if in_neg: 
                if word in vocab:
                    neg_words_seen.add(word+"_NOT")
                word += "_NOT"
            count_neg += 1
            bag_neg[word] = bag_neg.get(word, 0) + 1
            if word in negate: in_neg = True
    else:
        N_pos += 1
        for word in sequence.leaves():
            if word in punct: in_neg = False
            if in_neg: 
                if word in vocab:
                    neg_words_seen.add(word+"_NOT")
                word += "_NOT"
            count_pos += 1
            bag_pos[word] = bag_pos.get(word, 0) + 1
            if word in negate: in_neg = True

In [35]:
test_pred = [0]*len(test_set)
vocab_len = len(vocab) + 2
for sentence in range(len(test_set)):
    neg_score = 0
    pos_score = 0
    in_neg = False
    for word in test_set[sentence]:
        if word in punct: in_neg = False
        if in_neg:
            word += "_NOT"
        if word in vocab or word in neg_words_seen:
            neg_score += np.log( (bag_neg.get(word, 0) + 1) / (count_neg + vocab_len) )
            pos_score += np.log( (bag_pos.get(word, 0) + 1) / (count_pos + vocab_len) )
        if word in negate: in_neg = True
    
    neg_score += np.log((N_neg/N_doc))
    pos_score += np.log((N_pos/N_doc))
    if neg_score > pos_score:
        test_pred[sentence] = 0
    else:
        test_pred[sentence] = 1

test_verif = test.parsed_sents()
ratio = 0
for sentence in range(len(test_set)):
    y = int(test_verif[sentence].label())
    if (test_pred[sentence] == (y>=2)):
        ratio += 1
print((ratio/len(test_set))*100)

65.5
